In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pickle

In [ ]:
experiment_name = 'FDENSER10/FDENSER00'
NRUNS = 1

stats = []
for i in range (0, NRUNS):
	statistics_file = 'D:/experiments/' + experiment_name + '/statistics.pkl'
	with open(statistics_file, 'rb') as f:
		stats.append(pickle.load(f))

def hms(seconds):
	return time.strftime("%H:%M:%S", time.gmtime(seconds))


In [ ]:
stat = stats[0]

print(f"{experiment_name}: {stat.run_generation} generations")
print(f"{stat.evaluations_total} evaluations ({stat.evaluations_k_folds} for k-folds), {stat.evaluations_cache_hits} cache hits, {stat.evaluations_invalid} invalid")
print(f"runtime {hms(stat.run_time)}, evaluation time {hms(stat.eval_time)} (this run {hms(stat.eval_time_this_run)}), k-folds: {hms(stat.eval_time_k_folds)} (this run {hms(stat.eval_time_k_folds_this_run)})")

def plot_different_accuracies(ax, stat):
	ngenerations = stat.run_generation + 1
	xscale = np.arange(0, ngenerations)
	ax.set_title('Different Accuracy Measures', fontsize=25)
	ax.plot(stat.best.train_accuracy, label="training accuracy")
	ax.plot(stat.best.val_accuracy, label="validation accuracy")
	ax.plot(stat.best.accuracy, color='blue', label="(test) accuracy")
	ax.plot(stat.best.final_test_accuracy, label="final test accuracy")
	if len(stat.best.k_fold_accuracy):
		ax.plot(xscale, stat.best.k_fold_accuracy, '*-', color='cyan', label="avg K-fold accuracy")
		ax.errorbar(xscale, stat.best.k_fold_accuracy, yerr = stat.best.k_fold_accuracy_std, color='cyan', zorder=10)
		ax.plot(stat.best.k_fold_final_accuracy, label="avg K-fold final accuracy")
	ax.set_ylim(0.95, 1.0)
	ax.set_xlim(0, ngenerations)
	ax.legend(fontsize=12)
	ax.grid(True)

def plot_parameters(ax, stat):
	generation_parameters = np.array(stat.generation_parameters)
	(ngenerations, population_size) = generation_parameters.shape
	ax.set_title(f"Number of Parameters (best: {stat.best.parameters[-1]})", fontsize=25)
	for i in range(population_size):
		ax.plot(generation_parameters[:, i], 'o', markersize=4, color ='#C0C0C0', alpha=0.5, label = 'population', zorder=-32)
	ax.plot(stat.best.parameters, color='blue', label="best")
	ax.plot(stat.best_in_gen.parameters, color='magenta', alpha=0.7, label="best in generation")
	ax.set_xlim(0, ngenerations)
	ax.set_ylim(0, 100000)
	ax.yaxis.set_major_locator(ticker.MultipleLocator(10000))
	reduced_legend(ax, population_size, 2)

def plot_accuracy(ax, stat):
	generation_accuracy = np.array(stat.generation_accuracy)
	(ngenerations, population_size) = generation_accuracy.shape
	xscale = np.arange(0, ngenerations)
	ax.set_title(f"Accuracy (best: {stat.best.accuracy[-1]:.5f})", fontsize=25)
	for i in range(population_size):
		ax.plot(generation_accuracy[:, i], 'o', markersize=4, color ='#C0C0C0', alpha=0.5, label = 'population accuracy', zorder=-32)
	if len(stat.best.k_fold_accuracy):
		ax.plot(xscale, stat.best.k_fold_accuracy, '*-', color='cyan', alpha=1, label="K-folds of best")
		ax.errorbar(xscale, stat.best.k_fold_accuracy, yerr = stat.best.k_fold_accuracy_std, color='cyan', alpha=1, zorder=10)
	ax.plot(stat.best_in_gen.accuracy, '*-', color = 'magenta', alpha=0.5, label='best in generation')
	ax.plot(stat.best.accuracy, '*-', color = 'blue', alpha=0.5, label='best')
	ax.set_ylim(0.96, 1.0)
	ax.set_xlim(0, ngenerations)
	reduced_legend(ax, population_size, 2)
	
def plot_fitness(ax, stat):
	generation_fitness = np.array(stat.generation_fitness)
	(ngenerations, population_size) = generation_fitness.shape
	xscale = np.arange(0, ngenerations)
	ax.set_title(f"Fitness (best: {stat.best.fitness[-1]:.5f})", fontsize=25)
	for i in range(population_size):
		ax.plot(generation_fitness[:, i], 'o', markersize=4, color ='#C0C0C0', alpha=0.5, label = 'population fitness', zorder=-32)
	if len(stat.best.k_fold_fitness):
		ax.plot(xscale, stat.best.k_fold_fitness, '*-', color='cyan', alpha=1, label="K-folds of best")
		ax.errorbar(xscale, stat.best.k_fold_fitness, yerr = stat.best.k_fold_fitness_std, color='cyan', alpha=1, zorder=10)
	ax.plot(stat.best_in_gen.fitness, '*-', color = 'magenta', alpha=0.5, label='best in generation')
	ax.plot(stat.best.fitness, '*-', color = 'blue', alpha=0.5, label='best')
	ax.set_ylim(-2, 3)
	ax.set_xlim(0, ngenerations)
	reduced_legend(ax, population_size, 2)

# show label for 'Population accuracy' only once
def reduced_legend(ax, population_size, additional_entries=1):
	handles, labels = ax.get_legend_handles_labels()
	display = [0] + [i for i in range(population_size, population_size + additional_entries)]
	ax.legend([handle for i,handle in enumerate(handles) if i in display],
				[label for i,label in enumerate(labels) if i in display], loc = 'best', fontsize=15)
	ax.grid(True)

fig, ((ax, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20,14))

plot_accuracy(ax, stat)

plot_parameters(ax2, stat)

plot_fitness(ax3, stat)

plot_different_accuracies(ax4, stat)

In [ ]:
if stat.stepwidth_stats:
	fig, ax = plt.subplots()
	for (module, var, values) in stat.stepwidth_stats:
		ax.plot(values, label=f"{module}/{var}")
	ax.legend()